# Computer Vision - Project
imports

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import h5py
import requests
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
import tensorflow as tf
import h5py, requests, os
import matplotlib.patches as patches
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import LSTM
from keras.layers import Resizing
from tqdm import tqdm

In [2]:
# IMG_SIZE = (64, 64)
# IMG_SHAPE = IMG_SIZE + (3,)
# base_model = tf.keras.applications.resnet(input_shape=IMG_SHAPE,
#                                                include_top=False,
#                                                weights='imagenet')
# base_model.trainable = False

Constants

In [3]:
#ZIP_PATH = "https://drive.google.com/drive/folders/1jzHYpTwywUYA53nMGHVROSuVO14hEueq?usp=sharing/"
FILE_NAME ="SynthText_train.h5"
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)
tf.keras.backend.clear_session()
SIZE=227


In [4]:
def sort_points(points, center):
    # calculate the angle of each point from the center point
    angles = np.arctan2(points[:, 1] - center[1], points[:, 0] - center[0])
    # sort the points by angle
    sorted_points = points[np.argsort(angles)]
    return sorted_points

In [5]:
def draw_points(image, points, color=(255, 0, 0), radius=3):
    # create a copy of the image
    img = image.copy()
    # iterate over the points and draw them on the image
    for point in points:
        cv2.circle(img, tuple(map(int, point)), radius, color, -1)
    return img


In [6]:
def get_bb(img, bbs, indx):
    x1 = int(bbs[0,0,indx])
    y1 = int(bbs[1,0,indx])
    x2 = int(bbs[0,1,indx])
    y2 = int(bbs[1,1,indx])
    x3 = int(bbs[0,2,indx])
    y3 = int(bbs[1,2,indx])
    x4 = int(bbs[0,3,indx])
    y4 = int(bbs[1,3,indx])
    # calculate bounding rectangle
    top_left_x = max(0, min([x1,x2,x3,x4]))
    top_left_y = max(0, min([y1,y2,y3,y4]))
    bot_right_x = max(0, max([x1,x2,x3,x4]))
    bot_right_y = max(0, max([y1,y2,y3,y4]))
    # points = np.float32([[x1, y1], [x2, y2], [x3, y3], [x4, y4]])
    
    # create an empty image with the same shape as the input image
    # mask = np.zeros_like(img[:,:,0])
    # create a list of the bounding box points in the correct format
    # bounding_box = np.array([points], dtype=np.int32)
    # fill the area inside the bounding box with white
    # cv2.fillPoly(mask, bounding_box, 255)
    # apply the mask to the image
    ####res = cv2.bitwise_and(img, mask)
    # try1 = np.array(np.where(mask == 255, 255, 0), dtype=np.uint8)
    # print(x)
    # inv_mask = cv2.bitwise_not(mask)
    # res2 = cv2.bitwise_and(img, inv_mask)
    # bb2 = np.int32([[top_left_x, bot_right_y], [bot_right_x, bot_right_y],[bot_right_x,top_left_y], [top_left_x,top_left_y]])
    # print(bb2)
    # mask2 = np.zeros_like(img)
    # # bb2 = np.array(frame, dtype=np.int32)
    # cv2.fillPoly(mask2, bb2, (255, 255, 255))
    # mask = mask[top_left_y:bot_right_y+1, top_left_x:bot_right_x+1]
    res = img[top_left_y:bot_right_y+1, top_left_x:bot_right_x+1]

    #res = img[top_left_y:bot_right_y+1, top_left_x:bot_right_x+1]
    #flipping
    """"if(x2 < x1):
        res = cv2.flip(res, 1)
    if(y2 < y1):
        res = cv2.flip(res, 0)
    plt.imshow(mask)
    plt.show()
    print(mask.shape)"""
    return res

In [7]:
def font_to_num(font):
    if font == b'Alex Brush':
        return 0
    elif font == b'Titillium Web':
        return 1
    elif font == b'Sansation':
        return 2
    elif font == b'Open Sans':
        return 3
    else:
        return 4

In [8]:
def num_to_font(font):
    if font == 0:
        return b'Alex Brush'
    elif font == 1:
        return b'Titillium Web'
    elif font == 2:
        return b'Sansation'
    elif font == 3:
        return b'Open Sans'
    else:
        return b'Ubuntu Mono'

In [9]:
def print_label(set, index):
    line = set[index]
    max = np.argmax(line)
    print(num_to_font(max))

In [10]:
def print_font(max):
     print(num_to_font(max))

In [11]:
def print_photo_from_set(set_x, set_y, index, font):
    plt.imshow(set_x[index], cmap='gray')
    plt.show()
    print_font(font)

In [12]:
def prepare_img(img, bbs, index, size):
    cropped = get_bb(img, bbs, index)
    # _, cropped = cv2.threshold(cropped,127,255,cv2.THRESH_TRIANGLE)
    # print(cropped.shape)
    cropped = tf.image.resize(cropped, (size, size), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    # cropped = tf.image.rgb_to_grayscale(cropped)
    # # print(cropped.shape)
    cropped = tf.image.convert_image_dtype(cropped, tf.float32)
    # plt.imshow(cropped, cmap='gray')
    # plt.show()
    return cropped

In [13]:
from pathlib import Path
def create_dirs():
    Path('main_directory').mkdir(parents=True, exist_ok=True)
    Path('main_directory/Alex Brush').mkdir(parents=True, exist_ok=True)
    Path('main_directory/Titillium Web').mkdir(parents=True, exist_ok=True)
    Path('main_directory/Sansation').mkdir(parents=True, exist_ok=True)
    Path('main_directory/Open Sans').mkdir(parents=True, exist_ok=True)
    Path('main_directory/Ubuntu Mono').mkdir(parents=True, exist_ok=True)

In [14]:
def get_data_set(size: int):
    db = h5py.File(FILE_NAME, 'r')
    create_dirs()
    im_names = list(db['data'].keys())
    train_x = []
    train_y = []
    num = 0
    for i in tqdm(range(0, len(im_names)-1)):
        im = im_names[i]
        img  = db['data'][im][:]
        fonts = db['data'][im].attrs['font']
        txts = db['data'][im].attrs['txt']
        charBBs = db['data'][im].attrs['charBB']
        wordBBs = db['data'][im].attrs['wordBB']
        font_indx = 0 
        char_indx = 0
        # print(im)
        for j in range(0, len(txts)):
            cropped = prepare_img(img, wordBBs, j, size)
            path = 'main_directory/'+fonts[font_indx].decode('UTF-8')+'/'+im+'_'+str(num)+'.jpg' 
            # print(path)
            tf.keras.utils.save_img(path,cropped)
            num+=1
            # train_x.append(cropped)
            # train_y.append(font_to_num(fonts[font_indx]))
            
            # plt.imshow(cropped, cmap='gray')
            # plt.show()
            # print(fonts[font_indx])
            for k in range(0, len(txts[j])):
                cropped = prepare_img(img, charBBs, char_indx, size)
                path = 'main_directory/'+fonts[font_indx].decode('UTF-8')+'/'+im+'_'+str(num)+'.jpg'
                # print(path)
                tf.keras.utils.save_img(path,cropped)
                num+=1
                # train_x.append(cropped)
                # train_y.append(font_to_num(fonts[font_indx]))
                char_indx+=1
                # plt.imshow(cropped, cmap='gray')
                # plt.show()
                # print(fonts[font_indx])
            font_indx += len(txts[j])
    print(num)
    # return train_x, train_y

In [15]:
from download_h5py_db import download_h5py_db
if not Path("main_directory").exists():
    download_h5py_db()
    get_data_set(SIZE)
# train_x, train_y = 

In [16]:
from keras.preprocessing.image import ImageDataGenerator
def data_augmentation():
    datagen =  ImageDataGenerator(
        # horizontal_flip=True, rotation_range=10, fill_mode='reflect', 
        # shear_range=10, vertical_flip=False, channel_shift_range=1.0, brightness_range=(0.2, 0.8),
     rescale=1/255,dtype='float32',validation_split=0.9)
    # Path('augmented').mkdir(exist_ok=True)
    it = datagen.flow_from_directory('main_directory', batch_size=18, subset='training', class_mode='categorical',
    #save_to_dir='augmented',
     shuffle=True)
    val_it = datagen.flow_from_directory('main_directory', batch_size=18, subset='validation', class_mode='categorical',
    #save_to_dir='augmented',
     shuffle=True)
    return it, val_it, datagen

In [17]:
it, val_it, datagen = data_augmentation()

Found 3794 images belonging to 5 classes.
Found 34108 images belonging to 5 classes.


In [18]:
from tensorflow.keras.regularizers import l2

Regularizer = l2(0.003)

model = Sequential()
model.add(Conv2D(96, kernel_size=(11,11), strides= 4,
                        padding= 'valid', activation= 'relu',
                        input_shape= (SIZE,SIZE,3),
                        kernel_initializer= 'he_normal'))
model.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                        padding= 'valid', data_format= None))

model.add(Conv2D(256, kernel_size=(5,5), strides= 1,
                padding= 'same', activation= 'relu',
                kernel_initializer= 'he_normal'))
model.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                        padding= 'valid', data_format= None)) 

model.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                padding= 'same', activation= 'relu',
                kernel_initializer= 'he_normal'))

model.add(Conv2D(384, kernel_size=(3,3), strides= 1,
                padding= 'same', activation= 'relu',
                kernel_initializer= 'he_normal'))

model.add(Conv2D(256, kernel_size=(3,3), strides= 1,
                        padding= 'same', activation= 'relu',
                        kernel_initializer= 'he_normal'))

model.add(MaxPooling2D(pool_size=(3,3), strides= (2,2),
                        padding= 'valid', data_format= None))

model.add(Flatten())
model.add(Dense(4096, activation= 'relu'))
model.add(Dense(4096, activation= 'relu'))
model.add(Dense(1000, activation= 'relu', activity_regularizer=Regularizer, kernel_regularizer=Regularizer))
model.add(Dense(5, activation= 'softmax', activity_regularizer=Regularizer, kernel_regularizer=Regularizer))

model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=3e-6, amsgrad=True, epsilon=0.1),
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [19]:
def scheduler(epoch, lr):
    # if epoch < 5:
    #     return lr
    # else:
    return lr * 0.1 #tf.math.exp(-0.1)

In [20]:
# callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [21]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 256)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 13, 13, 384)       885120    
                                                                 
 conv2d_3 (Conv2D)           (None, 13, 13, 384)       1

In [22]:
# model.fit(X_train, Y_train, batch_size=16, epochs=20, verbose=1)
# datagen.fit(X_train)
model.fit(it, epochs=15, verbose=1 , shuffle=True
        #   ,callbacks=[callback]
          )

Epoch 1/15
211/211 [==============================] - 16s 59ms/step - loss: 6.7787 - accuracy: 0.2306
Epoch 2/15
211/211 [==============================] - 12s 55ms/step - loss: 6.5204 - accuracy: 0.2654
Epoch 3/15
211/211 [==============================] - 12s 55ms/step - loss: 6.4693 - accuracy: 0.2926
Epoch 4/15
211/211 [==============================] - 12s 55ms/step - loss: 6.4438 - accuracy: 0.3137
Epoch 5/15
211/211 [==============================] - 12s 55ms/step - loss: 6.4238 - accuracy: 0.3289
Epoch 6/15
211/211 [==============================] - 12s 55ms/step - loss: 6.4085 - accuracy: 0.3595
Epoch 7/15
211/211 [==============================] - 12s 55ms/step - loss: 6.3921 - accuracy: 0.3585
Epoch 8/15
211/211 [==============================] - 12s 55ms/step - loss: 6.3774 - accuracy: 0.3711
Epoch 9/15
211/211 [==============================] - 12s 56ms/step - loss: 6.3624 - accuracy: 0.3848
Epoch 10/15
211/211 [==============================] - 12s 55ms/step - loss: 6.346

In [23]:
score = model.evaluate(val_it)

1122/1895 [================>.............] - ETA: 23s - loss: 6.3303 - accuracy: 0.3801

KeyboardInterrupt: 

In [ ]:
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

In [ ]:
# 632/632 [==============================] - 150s 236ms/step - loss: 0.8984 - accuracy: 0.6251


In [ ]:
predict = model.predict(val_it, verbose=0)